In [199]:
import pandas as pd
import numpy as np
from py2neo import Node, Graph, Relationship
from neo4j import GraphDatabase, basic_auth

from keras.models import Model
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image

In [228]:
url = 'bolt://localhost:7687'
user = 'neo4j'
pwd = '1234'

graph = Graph(url, auth=('neo4j', pwd))
driver = GraphDatabase.driver(uri, auth = basic_auth(user,pwd))

In [136]:
graph.run('Match (n) DETACH DELETE n')

(No data)

In [137]:
# use the comment line if setting up database for the first time
# graph.run('CREATE CONSTRAINT n10s_unique_uri ON (r:Resource) ASSERT r.uri IS UNIQUE;')

In [138]:
graph.run('call n10s.graphconfig.init( {  handleMultival: "ARRAY" });')

param,value
handleVocabUris,SHORTEN
handleMultival,ARRAY
handleRDFTypes,LABELS


In [139]:
graph.run('CALL n10s.rdf.import.fetch("file:///D:\RMBI_MMKG_data\FB15K\FB15K_EntityTriples_Final.nt","N-Triples");')

terminationStatus,triplesLoaded,triplesParsed,namespaces,extraInfo,callParams
OK,592213,592213,{ns0: 'http://rdf.freebase.com/ns/'},,{}


In [218]:
graph.run('MATCH (n) RETURN count(*);')

count(*)
14953


In [192]:
img_des = pd.read_csv('D:\RMBI_MMKG_data\img_des.csv')
img_des

,LabelName,name,description
0,/m/027rn,Dominican Republic,The Dominican Republic is a country located on...
1,/m/017dcd,Mighty Morphin Power Rangers,Mighty Morphin Power Rangers is an American su...
2,/m/01sl1q,Michelle Rodriguez,Mayte Michelle Rodriguez is an American actres...
3,/m/0cnk2q,Australia national football team,The Australia men's national soccer team repre...
4,/m/02_j1w,Defender,"In the sport of association football, a defend..."
...,...,...,...
14179,/m/0dlj8q2,Le Monde's 100 Books of the Century,The 100 Books of the Century is a list of the ...
14180,/m/05zp8,Palace,"A palace is a grand residence, especially a ro..."
14181,/m/024030,Padma Vibhushan,The Padma Vibhushan is the second-highest civi...
14182,/m/02y_9cf,Ethiopian Empire,"The Ethiopian Empire, also formerly known by t..."


In [193]:
img_des['LabelName'] = img_des['LabelName'].str.replace('/m/','http://rdf.freebase.com/ns/m.')
img_des['name'] = img_des['name'].str.replace('\"','\\\"')
img_des['description'] = img_des['description'].str.replace('\"','\\\"')
img_des

,LabelName,name,description
0,http://rdf.freebase.com/ns/m.027rn,Dominican Republic,The Dominican Republic is a country located on...
1,http://rdf.freebase.com/ns/m.017dcd,Mighty Morphin Power Rangers,Mighty Morphin Power Rangers is an American su...
2,http://rdf.freebase.com/ns/m.01sl1q,Michelle Rodriguez,Mayte Michelle Rodriguez is an American actres...
3,http://rdf.freebase.com/ns/m.0cnk2q,Australia national football team,The Australia men's national soccer team repre...
4,http://rdf.freebase.com/ns/m.02_j1w,Defender,"In the sport of association football, a defend..."
...,...,...,...
14179,http://rdf.freebase.com/ns/m.0dlj8q2,Le Monde's 100 Books of the Century,The 100 Books of the Century is a list of the ...
14180,http://rdf.freebase.com/ns/m.05zp8,Palace,"A palace is a grand residence, especially a ro..."
14181,http://rdf.freebase.com/ns/m.024030,Padma Vibhushan,The Padma Vibhushan is the second-highest civi...
14182,http://rdf.freebase.com/ns/m.02y_9cf,Ethiopian Empire,"The Ethiopian Empire, also formerly known by t..."


In [194]:
for i in range(len(img_des)):
    graph.run('MATCH (a) WHERE a.uri = "%s" SET a.name = "%s", a.description = "%s";'% (img_des['LabelName'][i], img_des['name'][i], img_des['description'][i])) 

In [204]:
graph.run('Match (a:Resource) RETURN a.name as name, a.description as description LIMIT 3')

name,description
Blazing Saddles,"Blazing Saddles is a 1974 American Western black comedy film directed by Mel Brooks. Starring Cleavon Little and Gene Wilder, the film was written by Brooks, Andrew Bergman, Richard Pryor, Norman Steinberg, and Alan Uger, and was based on Bergman's story and draft."
C. J. Cherryh,"Carolyn Janice Cherry, better known by the pen name C. J. Cherryh, is an American writer of speculative fiction. She has written more than 80 books since the mid-1970s, including the Hugo Award-winning novels Downbelow Station and Cyteen, both set in her Alliance–Union universe, and her Foreigner universe."
Ronald D. Moore,"Ronald Dowl Moore is an American screenwriter and television producer. He is best known for his work on Star Trek; on the re-imagined Battlestar Galactica television series, for which he won a Peabody Award and an Emmy Award; and on Outlander, based on the novels of Diana Gabaldon."


In [205]:
ResNet = ResNet50(weights='imagenet', include_top=True, input_shape=(224, 224, 3))
for layer in ResNet.layers:
    layer.trainable = False
ResNet = Model(inputs=ResNet.inputs, outputs=ResNet.layers[-2].output)
ResNet.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_2[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                            

                                                                                                  
 conv2_block3_1_relu (Activatio  (None, 56, 56, 64)  0           ['conv2_block3_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv2_block3_2_conv (Conv2D)   (None, 56, 56, 64)   36928       ['conv2_block3_1_relu[0][0]']    
                                                                                                  
 conv2_block3_2_bn (BatchNormal  (None, 56, 56, 64)  256         ['conv2_block3_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv2_block3_2_relu (Activatio  (None, 56, 56, 64)  0           ['conv2_block3_2_bn[0][0]']      
 n)       

                                                                                                  
 conv3_block3_1_relu (Activatio  (None, 28, 28, 128)  0          ['conv3_block3_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv3_block3_2_conv (Conv2D)   (None, 28, 28, 128)  147584      ['conv3_block3_1_relu[0][0]']    
                                                                                                  
 conv3_block3_2_bn (BatchNormal  (None, 28, 28, 128)  512        ['conv3_block3_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv3_block3_2_relu (Activatio  (None, 28, 28, 128)  0          ['conv3_block3_2_bn[0][0]']      
 n)       

                                                                                                  
 conv4_block2_1_bn (BatchNormal  (None, 14, 14, 256)  1024       ['conv4_block2_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv4_block2_1_relu (Activatio  (None, 14, 14, 256)  0          ['conv4_block2_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv4_block2_2_conv (Conv2D)   (None, 14, 14, 256)  590080      ['conv4_block2_1_relu[0][0]']    
                                                                                                  
 conv4_block2_2_bn (BatchNormal  (None, 14, 14, 256)  1024       ['conv4_block2_2_conv[0][0]']    
 ization) 

 conv4_block5_1_conv (Conv2D)   (None, 14, 14, 256)  262400      ['conv4_block4_out[0][0]']       
                                                                                                  
 conv4_block5_1_bn (BatchNormal  (None, 14, 14, 256)  1024       ['conv4_block5_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv4_block5_1_relu (Activatio  (None, 14, 14, 256)  0          ['conv4_block5_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv4_block5_2_conv (Conv2D)   (None, 14, 14, 256)  590080      ['conv4_block5_1_relu[0][0]']    
                                                                                                  
 conv4_blo

                                                                  'conv5_block1_3_bn[0][0]']      
                                                                                                  
 conv5_block1_out (Activation)  (None, 7, 7, 2048)   0           ['conv5_block1_add[0][0]']       
                                                                                                  
 conv5_block2_1_conv (Conv2D)   (None, 7, 7, 512)    1049088     ['conv5_block1_out[0][0]']       
                                                                                                  
 conv5_block2_1_bn (BatchNormal  (None, 7, 7, 512)   2048        ['conv5_block2_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv5_block2_1_relu (Activatio  (None, 7, 7, 512)   0           ['conv5_block2_1_bn[0][0]']      
 n)       

In [229]:
x_img = []
img_name = []
uri = []
import os

for i in range(100):
    path_of_the_directory = os.path.join('D:\RMBI_MMKG_data\Bulk-Bing-Image-downloader', img_des['name'][i])
    ext = ('.jpg')
    for path, dirc, files in os.walk(path_of_the_directory):
        for name in files:
            if name.endswith(ext):
                
                try:
                    img_path = os.path.join(path, name)               
                    img = image.load_img(img_path, target_size=(224, 224))
                    img = image.img_to_array(img)
                
                    x_img.append(img)
                    img_name.append(name)
                    uri.append(img_des['LabelName'][i])
                except:
                    print(img_path)

In [230]:
x_img = np.array(x_img)

In [231]:
x_img = x_img / 255.0

In [232]:
img_emb = ResNet.predict(x_img)
img_emb.shape

(868, 2048)

In [233]:
img_emb = img_emb.tolist()

In [234]:
len(img_emb)

868

In [235]:
statements = ""

for i in range(len(img_emb)):
    statement = "MERGE (img_%d:Image {uri: '%s', name: '%s'}) SET img_%d.embedding = %s \n" % (i, uri[i], img_name[i], i, img_emb[i])
    statements += statement

In [236]:
session = driver.session()

In [237]:
trx = session.begin_transaction()

In [238]:
trx.run(statements)
trx.commit()

'FB:kcwQNWvC3anSQg61g3OAe7kCbsk9mpA='

In [239]:
re = graph.run('Match (n:Image) RETURN count(*)')
re.data()

[{'count(*)': 868}]

In [241]:
graph.run('Match (n:Image) RETURN n.uri as uri, n.name as name LIMIT 3')

uri,name
http://rdf.freebase.com/ns/m.027rn,img_1.jpg
http://rdf.freebase.com/ns/m.027rn,img_2.jpg
http://rdf.freebase.com/ns/m.027rn,img_3.jpg


In [242]:
img_des

,LabelName,name,description
0,http://rdf.freebase.com/ns/m.027rn,Dominican Republic,The Dominican Republic is a country located on...
1,http://rdf.freebase.com/ns/m.017dcd,Mighty Morphin Power Rangers,Mighty Morphin Power Rangers is an American su...
2,http://rdf.freebase.com/ns/m.01sl1q,Michelle Rodriguez,Mayte Michelle Rodriguez is an American actres...
3,http://rdf.freebase.com/ns/m.0cnk2q,Australia national football team,The Australia men's national soccer team repre...
4,http://rdf.freebase.com/ns/m.02_j1w,Defender,"In the sport of association football, a defend..."
...,...,...,...
14179,http://rdf.freebase.com/ns/m.0dlj8q2,Le Monde's 100 Books of the Century,The 100 Books of the Century is a list of the ...
14180,http://rdf.freebase.com/ns/m.05zp8,Palace,"A palace is a grand residence, especially a ro..."
14181,http://rdf.freebase.com/ns/m.024030,Padma Vibhushan,The Padma Vibhushan is the second-highest civi...
14182,http://rdf.freebase.com/ns/m.02y_9cf,Ethiopian Empire,"The Ethiopian Empire, also formerly known by t..."
